In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
import random
import sys
import os


# set seed
def seed_everything(SEED):
    np.random.seed(SEED)
    tf.random.set_seed(SEED)
    random.seed(SEED)

from util.TwoDPCA import *
from util.TTwoPCA import *
from util.data_process import *
from util.model import *
from util.waivelet import *
from util.SAE import *

def get_baseline_data(df,window_size = 60):
    data, price = data_imputation_without_feature_selection(df)
    data, price, sc = normalize(data, price)
    data, price = data_transform(data, price, window_size)
    X_train, X_test, y_train, y_test = split_data(data, price)
    X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
    X_test = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))
    xtrain, xVal, ytrain, yVal = split_data(X_train, y_train)
    return X_train, xVal, y_train, yVal,sc,X_test,y_test

def get_wt_pca_data(df,output_size = 200,window_size = 60):
    data, price = data_imputation(df)
    data, price, sc = normalize(data, price)
    data = WT(data)
    data, price = data_transform(data, price, window_size=60)
    data = PCA_transform(data, output_size)
    X_train, X_test, y_train, y_test = split_data(data, price)
    X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
    X_test = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))
    xtrain, xVal, ytrain, yVal = split_data(X_train, y_train)
    return X_train, xVal, y_train, yVal,sc,X_test,y_test

def get_wt_2dpca_data(df,output_size = 11,window_size = 60):
    data, price = data_imputation(df)
    data, price, sc = normalize(data, price)
    data = WT(data)
    data, price = data_transform_2d_pca(data, price, window_size=60)
    data = twoDPCA(data, 11)
    data = data.reshape(data.shape[0], -1)
    X_train, X_test, y_train, y_test = split_data(data, price)
    X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
    X_test = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))
    xtrain, xVal, ytrain, yVal = split_data(X_train, y_train)
    return X_train, xVal, y_train, yVal,sc,X_test,y_test

def get_wt_2d2dpca_data(df,row=16,col=10,window_size = 60):
    data, price = data_imputation(df)
    data, price, sc = normalize(data, price)
    data = WT(data)
    data, price = data_transform_2d_pca(data, price, window_size=60)
    data = ttwoDPCA(data, col, row)
    data = data.reshape(data.shape[0], -1)
    X_train, X_test, y_train, y_test = split_data(data, price)
    X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
    X_test = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))
    xtrain, xVal, ytrain, yVal = split_data(X_train, y_train)
    return X_train, xVal, y_train, yVal,sc,X_test,y_test

def get_wt_SAE_data(df,output_dim = 121,epoch=100,batch_size=256):
    data, price = data_imputation(df)
    data, price, sc = normalize(data, price)
    data = WT(data)
    data, price = data_transform(data, price, window_size=60)
    data = SAE(data)
    np.save("./data/SAE_data.npy",data)
    X_train, X_test, y_train, y_test = split_data(data, price)
    X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
    X_test = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))
    xtrain, xVal, ytrain, yVal = split_data(X_train, y_train)
    return X_train, xVal, y_train, yVal,sc,X_test,y_test

def load_SAE_data(df,window_size = 60):
    data, price = data_imputation(df)
    data, price, sc = normalize(data, price)
    data = np.load("./data/SAE_data.npy")
    X_train, X_test, y_train, y_test = split_data(data, price[window_size:])
    X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
    X_test = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))
    xtrain, xVal, ytrain, yVal = split_data(X_train, y_train)
    return X_train, xVal, y_train, yVal, sc, X_test, y_test

In [2]:
seed_everything(6000)
df = pd.read_csv("./data/addedfeatures.csv")
new_date = pd.to_datetime(df["date"], format='%Y%m%d', errors='ignore')
df["date"] = new_date
df.index = df.date
price = df["PRC"]

### BaseLine

In [3]:
xtrain, xVal, ytrain, yVal, sc, X_test, y_test = get_baseline_data(df)

/Users/fuyuqi/anaconda2/envs/notebook/lib/python3.7/site-packages/sklearn/utils/deprecation.py:66: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


In [4]:
train_LSTM_model(xtrain, ytrain, xVal, yVal, filepath="./model_weights/BASE_LSTM.h5")

Train on 3775 samples, validate on 755 samples
Epoch 1/100
3775/3775 [==============================] - 26s 7ms/sample - loss: 0.1918 - val_loss: 0.1905
Epoch 2/100
3775/3775 [==============================] - 1s 348us/sample - loss: 0.0455 - val_loss: 0.0289
Epoch 3/100
3775/3775 [==============================] - 2s 477us/sample - loss: 0.0278 - val_loss: 0.0114
Epoch 4/100
3775/3775 [==============================] - 1s 388us/sample - loss: 0.0232 - val_loss: 0.0086
Epoch 5/100
3775/3775 [==============================] - 1s 243us/sample - loss: 0.0213 - val_loss: 0.0100
Epoch 6/100
3775/3775 [==============================] - 1s 248us/sample - loss: 0.0203 - val_loss: 0.0102
Epoch 7/100
3775/3775 [==============================] - 1s 253us/sample - loss: 0.0210 - val_loss: 0.0092
Epoch 8/100
3775/3775 [==============================] - 1s 352us/sample - loss: 0.0196 - val_loss: 0.0060
Epoch 9/100
3775/3775 [==============================] - 1s 267us/sample - loss: 0.0183 - val_loss

3775/3775 [==============================] - 1s 352us/sample - loss: 0.0099 - val_loss: 0.0033
Epoch 77/100
3775/3775 [==============================] - 1s 370us/sample - loss: 0.0096 - val_loss: 0.0029
Epoch 78/100
3775/3775 [==============================] - 1s 338us/sample - loss: 0.0097 - val_loss: 0.0025
Epoch 79/100
3775/3775 [==============================] - 1s 318us/sample - loss: 0.0101 - val_loss: 0.0039
Epoch 80/100
3775/3775 [==============================] - 1s 264us/sample - loss: 0.0102 - val_loss: 0.0025
Epoch 81/100
3775/3775 [==============================] - 1s 353us/sample - loss: 0.0099 - val_loss: 0.0024
Epoch 82/100
3775/3775 [==============================] - 2s 412us/sample - loss: 0.0095 - val_loss: 0.0041
Epoch 83/100
3775/3775 [==============================] - 1s 323us/sample - loss: 0.0099 - val_loss: 0.0039
Epoch 84/100
3775/3775 [==============================] - 1s 328us/sample - loss: 0.0106 - val_loss: 0.0028
Epoch 85/100
3775/3775 [=================

In [5]:
train_RNN_model(xtrain, ytrain, xVal, yVal, filepath="./model_weights/BASE_RNN.h5")

Train on 3775 samples, validate on 755 samples
Epoch 1/100
3775/3775 [==============================] - 21s 5ms/sample - loss: 0.7163 - val_loss: 0.0259
Epoch 2/100
3775/3775 [==============================] - 1s 303us/sample - loss: 0.4475 - val_loss: 0.0215
Epoch 3/100
3775/3775 [==============================] - 1s 324us/sample - loss: 0.3758 - val_loss: 0.0476
Epoch 4/100
3775/3775 [==============================] - 1s 218us/sample - loss: 0.2872 - val_loss: 0.0164
Epoch 5/100
3775/3775 [==============================] - 1s 347us/sample - loss: 0.2775 - val_loss: 0.0190
Epoch 6/100
3775/3775 [==============================] - 1s 266us/sample - loss: 0.2369 - val_loss: 0.0159
Epoch 7/100
3775/3775 [==============================] - 1s 211us/sample - loss: 0.1995 - val_loss: 0.0127
Epoch 8/100
3775/3775 [==============================] - 1s 156us/sample - loss: 0.1719 - val_loss: 0.0508
Epoch 9/100
3775/3775 [==============================] - 1s 140us/sample - loss: 0.1510 - val_loss

3775/3775 [==============================] - 1s 159us/sample - loss: 0.0160 - val_loss: 0.0077
Epoch 77/100
3775/3775 [==============================] - 1s 257us/sample - loss: 0.0159 - val_loss: 0.0062
Epoch 78/100
3775/3775 [==============================] - 1s 227us/sample - loss: 0.0155 - val_loss: 0.0072
Epoch 79/100
3775/3775 [==============================] - 1s 164us/sample - loss: 0.0147 - val_loss: 0.0069
Epoch 80/100
3775/3775 [==============================] - 1s 172us/sample - loss: 0.0158 - val_loss: 0.0080
Epoch 81/100
3775/3775 [==============================] - 1s 262us/sample - loss: 0.0154 - val_loss: 0.0095
Epoch 82/100
3775/3775 [==============================] - 1s 385us/sample - loss: 0.0158 - val_loss: 0.0073
Epoch 83/100
3775/3775 [==============================] - 1s 190us/sample - loss: 0.0160 - val_loss: 0.0065
Epoch 84/100
3775/3775 [==============================] - 1s 222us/sample - loss: 0.0146 - val_loss: 0.0072
Epoch 85/100
3775/3775 [=================

In [6]:
train_GRU_model(xtrain, ytrain, xVal, yVal, filepath="./model_weights/BASE_GRU.h5")

Train on 3775 samples, validate on 755 samples
Epoch 1/100
3775/3775 [==============================] - 35s 9ms/sample - loss: 0.0971 - val_loss: 0.0160
Epoch 2/100
3775/3775 [==============================] - 1s 333us/sample - loss: 0.0400 - val_loss: 0.0068
Epoch 3/100
3775/3775 [==============================] - 1s 347us/sample - loss: 0.0359 - val_loss: 0.0180
Epoch 4/100
3775/3775 [==============================] - 2s 442us/sample - loss: 0.0311 - val_loss: 0.0140
Epoch 5/100
3775/3775 [==============================] - 1s 376us/sample - loss: 0.0318 - val_loss: 0.0723
Epoch 6/100
3775/3775 [==============================] - 1s 335us/sample - loss: 0.0294 - val_loss: 0.0185
Epoch 7/100
3775/3775 [==============================] - 1s 386us/sample - loss: 0.0259 - val_loss: 0.0129
Epoch 8/100
3775/3775 [==============================] - 1s 378us/sample - loss: 0.0269 - val_loss: 0.0109
Epoch 9/100
3775/3775 [==============================] - 1s 364us/sample - loss: 0.0253 - val_loss

3775/3775 [==============================] - 1s 217us/sample - loss: 0.0105 - val_loss: 0.0032
Epoch 77/100
3775/3775 [==============================] - 1s 317us/sample - loss: 0.0105 - val_loss: 0.0020
Epoch 78/100
3775/3775 [==============================] - 1s 301us/sample - loss: 0.0103 - val_loss: 0.0017
Epoch 79/100
3775/3775 [==============================] - 1s 336us/sample - loss: 0.0116 - val_loss: 0.0024
Epoch 80/100
3775/3775 [==============================] - 1s 290us/sample - loss: 0.0098 - val_loss: 0.0022
Epoch 81/100
3775/3775 [==============================] - 1s 347us/sample - loss: 0.0102 - val_loss: 0.0024
Epoch 82/100
3775/3775 [==============================] - 2s 400us/sample - loss: 0.0103 - val_loss: 0.0021
Epoch 83/100
3775/3775 [==============================] - 2s 406us/sample - loss: 0.0104 - val_loss: 0.0033
Epoch 84/100
3775/3775 [==============================] - 1s 377us/sample - loss: 0.0098 - val_loss: 0.0022
Epoch 85/100
3775/3775 [=================

### SAE

In [7]:
xtrain, xVal, ytrain, yVal,sc,X_test,y_test = get_wt_SAE_data(df)

/Users/fuyuqi/anaconda2/envs/notebook/lib/python3.7/site-packages/sklearn/utils/deprecation.py:66: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


Train on 4719 samples
Epoch 1/100
4719/4719 [==============================] - 2s 476us/sample - loss: 0.2225
Epoch 2/100
4719/4719 [==============================] - 1s 134us/sample - loss: 0.0789
Epoch 3/100
4719/4719 [==============================] - 1s 128us/sample - loss: 0.0427
Epoch 4/100
4719/4719 [==============================] - 1s 163us/sample - loss: 0.0362
Epoch 5/100
4719/4719 [==============================] - 1s 159us/sample - loss: 0.0303
Epoch 6/100
4719/4719 [==============================] - 1s 166us/sample - loss: 0.0252
Epoch 7/100
4719/4719 [==============================] - 1s 166us/sample - loss: 0.0206
Epoch 8/100
4719/4719 [==============================] - 1s 143us/sample - loss: 0.0170
Epoch 9/100
4719/4719 [==============================] - 1s 181us/sample - loss: 0.0147
Epoch 10/100
4719/4719 [==============================] - 1s 255us/sample - loss: 0.0134
Epoch 11/100
4719/4719 [==============================] - 1s 247us/sample - loss: 0.0122
Epoch 12

Epoch 93/100
4719/4719 [==============================] - 1s 173us/sample - loss: 0.0036
Epoch 94/100
4719/4719 [==============================] - 1s 176us/sample - loss: 0.0035
Epoch 95/100
4719/4719 [==============================] - 1s 169us/sample - loss: 0.0035
Epoch 96/100
4719/4719 [==============================] - 1s 167us/sample - loss: 0.0035
Epoch 97/100
4719/4719 [==============================] - 1s 142us/sample - loss: 0.0040
Epoch 98/100
4719/4719 [==============================] - 1s 134us/sample - loss: 0.0039
Epoch 99/100
4719/4719 [==============================] - 1s 149us/sample - loss: 0.0036
Epoch 100/100
4719/4719 [==============================] - 1s 199us/sample - loss: 0.0035
Train on 4719 samples
Epoch 1/100
4719/4719 [==============================] - 2s 484us/sample - loss: 0.1664
Epoch 2/100
4719/4719 [==============================] - 1s 138us/sample - loss: 0.0701
Epoch 3/100
4719/4719 [==============================] - 1s 155us/sample - loss: 0.0407
E

4719/4719 [==============================] - 1s 121us/sample - loss: 0.0013
Epoch 86/100
4719/4719 [==============================] - 1s 112us/sample - loss: 0.0016
Epoch 87/100
4719/4719 [==============================] - 1s 113us/sample - loss: 0.0013
Epoch 88/100
4719/4719 [==============================] - 1s 149us/sample - loss: 0.0012
Epoch 89/100
4719/4719 [==============================] - 1s 178us/sample - loss: 0.0011
Epoch 90/100
4719/4719 [==============================] - 1s 161us/sample - loss: 0.0011
Epoch 91/100
4719/4719 [==============================] - 1s 185us/sample - loss: 0.0011
Epoch 92/100
4719/4719 [==============================] - 1s 184us/sample - loss: 0.0010
Epoch 93/100
4719/4719 [==============================] - 1s 234us/sample - loss: 0.0011
Epoch 94/100
4719/4719 [==============================] - 1s 189us/sample - loss: 0.0011
Epoch 95/100
4719/4719 [==============================] - 1s 167us/sample - loss: 0.0012
Epoch 96/100
4719/4719 [==========

4719/4719 [==============================] - 1s 115us/sample - loss: 5.3195e-04
Epoch 77/100
4719/4719 [==============================] - 0s 106us/sample - loss: 7.6215e-04
Epoch 78/100
4719/4719 [==============================] - 0s 102us/sample - loss: 8.9060e-04
Epoch 79/100
4719/4719 [==============================] - 0s 95us/sample - loss: 6.3212e-04
Epoch 80/100
4719/4719 [==============================] - 0s 95us/sample - loss: 5.9189e-04
Epoch 81/100
4719/4719 [==============================] - 1s 113us/sample - loss: 5.3878e-04
Epoch 82/100
4719/4719 [==============================] - 0s 105us/sample - loss: 5.1507e-04
Epoch 83/100
4719/4719 [==============================] - 0s 101us/sample - loss: 4.9869e-04
Epoch 84/100
4719/4719 [==============================] - 0s 93us/sample - loss: 4.8779e-04
Epoch 85/100
4719/4719 [==============================] - 0s 85us/sample - loss: 4.8052e-04
Epoch 86/100
4719/4719 [==============================] - 0s 91us/sample - loss: 4.8598

4719/4719 [==============================] - 0s 77us/sample - loss: 7.1076e-04
Epoch 67/100
4719/4719 [==============================] - 0s 65us/sample - loss: 4.8854e-04
Epoch 68/100
4719/4719 [==============================] - 0s 74us/sample - loss: 4.6470e-04
Epoch 69/100
4719/4719 [==============================] - 0s 67us/sample - loss: 4.2818e-04
Epoch 70/100
4719/4719 [==============================] - 0s 69us/sample - loss: 4.1725e-04
Epoch 71/100
4719/4719 [==============================] - 0s 72us/sample - loss: 4.0863e-04
Epoch 72/100
4719/4719 [==============================] - 0s 67us/sample - loss: 4.3593e-04
Epoch 73/100
4719/4719 [==============================] - 0s 59us/sample - loss: 5.0002e-04
Epoch 74/100
4719/4719 [==============================] - 0s 65us/sample - loss: 4.2266e-04
Epoch 75/100
4719/4719 [==============================] - 0s 86us/sample - loss: 3.9768e-04
Epoch 76/100
4719/4719 [==============================] - 0s 81us/sample - loss: 4.1812e-04
E

In [8]:
train_LSTM_model(xtrain, ytrain, xVal, yVal, filepath = "./model_weights/AE_LSTM.h5")

Train on 3775 samples, validate on 755 samples
Epoch 1/100
3775/3775 [==============================] - 22s 6ms/sample - loss: 0.2334 - val_loss: 0.3154
Epoch 2/100
3775/3775 [==============================] - 1s 221us/sample - loss: 0.0621 - val_loss: 0.0369
Epoch 3/100
3775/3775 [==============================] - 1s 186us/sample - loss: 0.0286 - val_loss: 0.0207
Epoch 4/100
3775/3775 [==============================] - 1s 188us/sample - loss: 0.0241 - val_loss: 0.0158
Epoch 5/100
3775/3775 [==============================] - 1s 198us/sample - loss: 0.0228 - val_loss: 0.0146
Epoch 6/100
3775/3775 [==============================] - 1s 184us/sample - loss: 0.0213 - val_loss: 0.0072
Epoch 7/100
3775/3775 [==============================] - 1s 216us/sample - loss: 0.0184 - val_loss: 0.0064
Epoch 8/100
3775/3775 [==============================] - 1s 203us/sample - loss: 0.0192 - val_loss: 0.0051
Epoch 9/100
3775/3775 [==============================] - 1s 193us/sample - loss: 0.0185 - val_loss

3775/3775 [==============================] - 1s 153us/sample - loss: 0.0115 - val_loss: 0.0048
Epoch 77/100
3775/3775 [==============================] - 1s 162us/sample - loss: 0.0115 - val_loss: 0.0059
Epoch 78/100
3775/3775 [==============================] - 1s 256us/sample - loss: 0.0109 - val_loss: 0.0041
Epoch 79/100
3775/3775 [==============================] - 1s 189us/sample - loss: 0.0115 - val_loss: 0.0096
Epoch 80/100
3775/3775 [==============================] - 1s 217us/sample - loss: 0.0108 - val_loss: 0.0047
Epoch 81/100
3775/3775 [==============================] - 1s 295us/sample - loss: 0.0117 - val_loss: 0.0046
Epoch 82/100
3775/3775 [==============================] - 1s 319us/sample - loss: 0.0118 - val_loss: 0.0057
Epoch 83/100
3775/3775 [==============================] - 1s 224us/sample - loss: 0.0112 - val_loss: 0.0043
Epoch 84/100
3775/3775 [==============================] - 1s 238us/sample - loss: 0.0107 - val_loss: 0.0053
Epoch 85/100
3775/3775 [=================

In [9]:
train_RNN_model(xtrain, ytrain, xVal, yVal, filepath="./model_weights/AE_RNN.h5")

Train on 3775 samples, validate on 755 samples
Epoch 1/100
3775/3775 [==============================] - 9s 2ms/sample - loss: 0.4763 - val_loss: 0.0189
Epoch 2/100
3775/3775 [==============================] - 0s 98us/sample - loss: 0.2026 - val_loss: 0.0388
Epoch 3/100
3775/3775 [==============================] - 0s 87us/sample - loss: 0.1509 - val_loss: 0.0215
Epoch 4/100
3775/3775 [==============================] - 0s 103us/sample - loss: 0.1190 - val_loss: 0.0098
Epoch 5/100
3775/3775 [==============================] - 0s 84us/sample - loss: 0.1005 - val_loss: 0.0413
Epoch 6/100
3775/3775 [==============================] - 0s 91us/sample - loss: 0.0803 - val_loss: 0.0087
Epoch 7/100
3775/3775 [==============================] - 0s 112us/sample - loss: 0.0767 - val_loss: 0.0072
Epoch 8/100
3775/3775 [==============================] - 0s 90us/sample - loss: 0.0671 - val_loss: 0.0063
Epoch 9/100
3775/3775 [==============================] - 0s 98us/sample - loss: 0.0570 - val_loss: 0.061

3775/3775 [==============================] - 0s 86us/sample - loss: 0.0166 - val_loss: 0.0091
Epoch 78/100
3775/3775 [==============================] - 0s 85us/sample - loss: 0.0157 - val_loss: 0.0071
Epoch 79/100
3775/3775 [==============================] - 0s 86us/sample - loss: 0.0174 - val_loss: 0.0070
Epoch 80/100
3775/3775 [==============================] - 0s 88us/sample - loss: 0.0169 - val_loss: 0.0083
Epoch 81/100
3775/3775 [==============================] - 0s 86us/sample - loss: 0.0170 - val_loss: 0.0091
Epoch 82/100
3775/3775 [==============================] - 0s 96us/sample - loss: 0.0168 - val_loss: 0.0082
Epoch 83/100
3775/3775 [==============================] - 0s 102us/sample - loss: 0.0165 - val_loss: 0.0089
Epoch 84/100
3775/3775 [==============================] - 0s 87us/sample - loss: 0.0166 - val_loss: 0.0073
Epoch 85/100
3775/3775 [==============================] - 0s 84us/sample - loss: 0.0162 - val_loss: 0.0088
Epoch 86/100
3775/3775 [=========================

In [10]:
train_GRU_model(xtrain, ytrain, xVal, yVal, filepath="./model_weights/AE_GRU.h5")

Train on 3775 samples, validate on 755 samples
Epoch 1/100
3775/3775 [==============================] - 16s 4ms/sample - loss: 0.1271 - val_loss: 0.0179
Epoch 2/100
3775/3775 [==============================] - 1s 167us/sample - loss: 0.0344 - val_loss: 0.0174
Epoch 3/100
3775/3775 [==============================] - 1s 147us/sample - loss: 0.0327 - val_loss: 0.0075
Epoch 4/100
3775/3775 [==============================] - 1s 155us/sample - loss: 0.0269 - val_loss: 0.0062
Epoch 5/100
3775/3775 [==============================] - 1s 137us/sample - loss: 0.0249 - val_loss: 0.0155
Epoch 6/100
3775/3775 [==============================] - 1s 154us/sample - loss: 0.0252 - val_loss: 0.0051
Epoch 7/100
3775/3775 [==============================] - 1s 141us/sample - loss: 0.0226 - val_loss: 0.0067
Epoch 8/100
3775/3775 [==============================] - 1s 168us/sample - loss: 0.0229 - val_loss: 0.0071
Epoch 9/100
3775/3775 [==============================] - 1s 146us/sample - loss: 0.0224 - val_loss

3775/3775 [==============================] - 1s 363us/sample - loss: 0.0117 - val_loss: 0.0053
Epoch 77/100
3775/3775 [==============================] - 1s 217us/sample - loss: 0.0129 - val_loss: 0.0071
Epoch 78/100
3775/3775 [==============================] - 1s 197us/sample - loss: 0.0120 - val_loss: 0.0062
Epoch 79/100
3775/3775 [==============================] - 1s 316us/sample - loss: 0.0125 - val_loss: 0.0082
Epoch 80/100
3775/3775 [==============================] - 1s 340us/sample - loss: 0.0130 - val_loss: 0.0052
Epoch 81/100
3775/3775 [==============================] - 1s 234us/sample - loss: 0.0122 - val_loss: 0.0053
Epoch 82/100
3775/3775 [==============================] - 1s 255us/sample - loss: 0.0120 - val_loss: 0.0074
Epoch 83/100
3775/3775 [==============================] - 1s 297us/sample - loss: 0.0121 - val_loss: 0.0063
Epoch 84/100
3775/3775 [==============================] - 1s 279us/sample - loss: 0.0120 - val_loss: 0.0056
Epoch 85/100
3775/3775 [=================

### PCA

In [11]:
xtrain, xVal, ytrain, yVal,sc,X_test,y_test = get_wt_pca_data(df)

/Users/fuyuqi/anaconda2/envs/notebook/lib/python3.7/site-packages/sklearn/utils/deprecation.py:66: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


In [12]:
train_LSTM_model(xtrain, ytrain, xVal, yVal, filepath="./model_weights/PCA_LSTM.h5")

Train on 3775 samples, validate on 755 samples
Epoch 1/100
3775/3775 [==============================] - 24s 6ms/sample - loss: 0.2424 - val_loss: 0.3530
Epoch 2/100
3775/3775 [==============================] - 1s 189us/sample - loss: 0.0708 - val_loss: 0.0305
Epoch 3/100
3775/3775 [==============================] - 1s 186us/sample - loss: 0.0242 - val_loss: 0.0117
Epoch 4/100
3775/3775 [==============================] - 1s 177us/sample - loss: 0.0209 - val_loss: 0.0087
Epoch 5/100
3775/3775 [==============================] - 1s 163us/sample - loss: 0.0193 - val_loss: 0.0073
Epoch 6/100
3775/3775 [==============================] - 1s 203us/sample - loss: 0.0184 - val_loss: 0.0057
Epoch 7/100
3775/3775 [==============================] - 1s 158us/sample - loss: 0.0172 - val_loss: 0.0071
Epoch 8/100
3775/3775 [==============================] - 1s 213us/sample - loss: 0.0169 - val_loss: 0.0050
Epoch 9/100
3775/3775 [==============================] - 1s 217us/sample - loss: 0.0161 - val_loss

3775/3775 [==============================] - 1s 148us/sample - loss: 0.0096 - val_loss: 0.0033
Epoch 77/100
3775/3775 [==============================] - 1s 161us/sample - loss: 0.0096 - val_loss: 0.0040
Epoch 78/100
3775/3775 [==============================] - 1s 148us/sample - loss: 0.0097 - val_loss: 0.0031
Epoch 79/100
3775/3775 [==============================] - 1s 145us/sample - loss: 0.0097 - val_loss: 0.0059
Epoch 80/100
3775/3775 [==============================] - 1s 150us/sample - loss: 0.0100 - val_loss: 0.0032
Epoch 81/100
3775/3775 [==============================] - 1s 160us/sample - loss: 0.0101 - val_loss: 0.0029
Epoch 82/100
3775/3775 [==============================] - 1s 166us/sample - loss: 0.0099 - val_loss: 0.0028
Epoch 83/100
3775/3775 [==============================] - 1s 156us/sample - loss: 0.0097 - val_loss: 0.0030
Epoch 84/100
3775/3775 [==============================] - 1s 158us/sample - loss: 0.0104 - val_loss: 0.0032
Epoch 85/100
3775/3775 [=================

In [13]:
train_RNN_model(xtrain, ytrain, xVal, yVal, filepath="./model_weights/PCA_RNN.h5")

Train on 3775 samples, validate on 755 samples
Epoch 1/100
3775/3775 [==============================] - 17s 5ms/sample - loss: 0.6131 - val_loss: 0.0229
Epoch 2/100
3775/3775 [==============================] - 1s 161us/sample - loss: 0.3890 - val_loss: 0.0593
Epoch 3/100
3775/3775 [==============================] - 1s 263us/sample - loss: 0.3199 - val_loss: 0.0332
Epoch 4/100
3775/3775 [==============================] - 1s 240us/sample - loss: 0.2629 - val_loss: 0.0155
Epoch 5/100
3775/3775 [==============================] - 1s 194us/sample - loss: 0.2322 - val_loss: 0.0166
Epoch 6/100
3775/3775 [==============================] - 1s 208us/sample - loss: 0.1890 - val_loss: 0.0295
Epoch 7/100
3775/3775 [==============================] - 1s 182us/sample - loss: 0.1703 - val_loss: 0.0192
Epoch 8/100
3775/3775 [==============================] - 1s 170us/sample - loss: 0.1443 - val_loss: 0.0337
Epoch 9/100
3775/3775 [==============================] - 1s 206us/sample - loss: 0.1258 - val_loss

Epoch 77/100
3775/3775 [==============================] - 0s 81us/sample - loss: 0.0148 - val_loss: 0.0060
Epoch 78/100
3775/3775 [==============================] - 0s 81us/sample - loss: 0.0140 - val_loss: 0.0064
Epoch 79/100
3775/3775 [==============================] - 0s 96us/sample - loss: 0.0128 - val_loss: 0.0106
Epoch 80/100
3775/3775 [==============================] - 0s 108us/sample - loss: 0.0152 - val_loss: 0.0095
Epoch 81/100
3775/3775 [==============================] - 1s 141us/sample - loss: 0.0136 - val_loss: 0.0083
Epoch 82/100
3775/3775 [==============================] - 1s 146us/sample - loss: 0.0139 - val_loss: 0.0055
Epoch 83/100
3775/3775 [==============================] - 0s 87us/sample - loss: 0.0146 - val_loss: 0.0053
Epoch 84/100
3775/3775 [==============================] - 0s 88us/sample - loss: 0.0139 - val_loss: 0.0100
Epoch 85/100
3775/3775 [==============================] - 0s 89us/sample - loss: 0.0141 - val_loss: 0.0056
Epoch 86/100
3775/3775 [==========

In [14]:
train_GRU_model(xtrain, ytrain, xVal, yVal, filepath="./model_weights/PCA_GRU.h5")

Train on 3775 samples, validate on 755 samples
Epoch 1/100
3775/3775 [==============================] - 17s 4ms/sample - loss: 0.1188 - val_loss: 0.0147
Epoch 2/100
3775/3775 [==============================] - 1s 150us/sample - loss: 0.0341 - val_loss: 0.0103
Epoch 3/100
3775/3775 [==============================] - 1s 153us/sample - loss: 0.0291 - val_loss: 0.0057
Epoch 4/100
3775/3775 [==============================] - 1s 141us/sample - loss: 0.0264 - val_loss: 0.0184
Epoch 5/100
3775/3775 [==============================] - 1s 136us/sample - loss: 0.0243 - val_loss: 0.0082
Epoch 6/100
3775/3775 [==============================] - 1s 148us/sample - loss: 0.0243 - val_loss: 0.0043
Epoch 7/100
3775/3775 [==============================] - 1s 149us/sample - loss: 0.0215 - val_loss: 0.0024
Epoch 8/100
3775/3775 [==============================] - 1s 156us/sample - loss: 0.0209 - val_loss: 0.0024
Epoch 9/100
3775/3775 [==============================] - 1s 141us/sample - loss: 0.0205 - val_loss

3775/3775 [==============================] - 1s 148us/sample - loss: 0.0102 - val_loss: 0.0035
Epoch 77/100
3775/3775 [==============================] - 1s 142us/sample - loss: 0.0109 - val_loss: 0.0033
Epoch 78/100
3775/3775 [==============================] - 1s 140us/sample - loss: 0.0104 - val_loss: 0.0037
Epoch 79/100
3775/3775 [==============================] - 1s 218us/sample - loss: 0.0116 - val_loss: 0.0038
Epoch 80/100
3775/3775 [==============================] - 1s 198us/sample - loss: 0.0102 - val_loss: 0.0055
Epoch 81/100
3775/3775 [==============================] - 1s 137us/sample - loss: 0.0107 - val_loss: 0.0043
Epoch 82/100
3775/3775 [==============================] - 1s 142us/sample - loss: 0.0104 - val_loss: 0.0053
Epoch 83/100
3775/3775 [==============================] - 1s 151us/sample - loss: 0.0106 - val_loss: 0.0055
Epoch 84/100
3775/3775 [==============================] - 1s 143us/sample - loss: 0.0099 - val_loss: 0.0037
Epoch 85/100
3775/3775 [=================

### 2DPCA

In [15]:
xtrain, xVal, ytrain, yVal,sc,X_test,y_test = get_wt_2dpca_data(df)

(14, 11)


/Users/fuyuqi/anaconda2/envs/notebook/lib/python3.7/site-packages/sklearn/utils/deprecation.py:66: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


In [16]:
train_LSTM_model(xtrain, ytrain, xVal, yVal, filepath="./model_weights/2dPCA_LSTM.h5")

Train on 3775 samples, validate on 755 samples
Epoch 1/100
3775/3775 [==============================] - 16s 4ms/sample - loss: 0.1938 - val_loss: 0.2060
Epoch 2/100
3775/3775 [==============================] - 1s 179us/sample - loss: 0.0468 - val_loss: 0.0232
Epoch 3/100
3775/3775 [==============================] - 1s 167us/sample - loss: 0.0291 - val_loss: 0.0087
Epoch 4/100
3775/3775 [==============================] - 1s 188us/sample - loss: 0.0254 - val_loss: 0.0091
Epoch 5/100
3775/3775 [==============================] - 1s 170us/sample - loss: 0.0230 - val_loss: 0.0077
Epoch 6/100
3775/3775 [==============================] - 1s 174us/sample - loss: 0.0229 - val_loss: 0.0063
Epoch 7/100
3775/3775 [==============================] - 1s 174us/sample - loss: 0.0223 - val_loss: 0.0061
Epoch 8/100
3775/3775 [==============================] - 1s 168us/sample - loss: 0.0206 - val_loss: 0.0054
Epoch 9/100
3775/3775 [==============================] - 1s 168us/sample - loss: 0.0217 - val_loss

3775/3775 [==============================] - 1s 160us/sample - loss: 0.0107 - val_loss: 0.0056
Epoch 77/100
3775/3775 [==============================] - 1s 162us/sample - loss: 0.0113 - val_loss: 0.0061
Epoch 78/100
3775/3775 [==============================] - 1s 194us/sample - loss: 0.0107 - val_loss: 0.0043
Epoch 79/100
3775/3775 [==============================] - 1s 158us/sample - loss: 0.0114 - val_loss: 0.0048
Epoch 80/100
3775/3775 [==============================] - 1s 157us/sample - loss: 0.0119 - val_loss: 0.0044
Epoch 81/100
3775/3775 [==============================] - 1s 158us/sample - loss: 0.0113 - val_loss: 0.0041
Epoch 82/100
3775/3775 [==============================] - 1s 170us/sample - loss: 0.0108 - val_loss: 0.0054
Epoch 83/100
3775/3775 [==============================] - 1s 168us/sample - loss: 0.0115 - val_loss: 0.0040
Epoch 84/100
3775/3775 [==============================] - 1s 184us/sample - loss: 0.0116 - val_loss: 0.0043
Epoch 85/100
3775/3775 [=================

In [17]:
train_RNN_model(xtrain, ytrain, xVal, yVal, filepath="./model_weights/2dPCA_RNN.h5")

Train on 3775 samples, validate on 755 samples
Epoch 1/100
3775/3775 [==============================] - 8s 2ms/sample - loss: 0.5649 - val_loss: 0.0102
Epoch 2/100
3775/3775 [==============================] - 0s 104us/sample - loss: 0.3333 - val_loss: 0.0085
Epoch 3/100
3775/3775 [==============================] - 0s 95us/sample - loss: 0.2386 - val_loss: 0.0205
Epoch 4/100
3775/3775 [==============================] - 0s 96us/sample - loss: 0.2215 - val_loss: 0.0085
Epoch 5/100
3775/3775 [==============================] - 0s 92us/sample - loss: 0.1788 - val_loss: 0.0094
Epoch 6/100
3775/3775 [==============================] - 0s 98us/sample - loss: 0.1554 - val_loss: 0.0059
Epoch 7/100
3775/3775 [==============================] - 0s 89us/sample - loss: 0.1286 - val_loss: 0.0135
Epoch 8/100
3775/3775 [==============================] - 0s 103us/sample - loss: 0.1192 - val_loss: 0.0100
Epoch 9/100
3775/3775 [==============================] - 0s 121us/sample - loss: 0.1019 - val_loss: 0.00

Epoch 77/100
3775/3775 [==============================] - 1s 150us/sample - loss: 0.0173 - val_loss: 0.0105
Epoch 78/100
3775/3775 [==============================] - 0s 105us/sample - loss: 0.0182 - val_loss: 0.0105
Epoch 79/100
3775/3775 [==============================] - 0s 113us/sample - loss: 0.0182 - val_loss: 0.0097
Epoch 80/100
3775/3775 [==============================] - 0s 99us/sample - loss: 0.0183 - val_loss: 0.0095
Epoch 81/100
3775/3775 [==============================] - 1s 138us/sample - loss: 0.0184 - val_loss: 0.0161
Epoch 82/100
3775/3775 [==============================] - 0s 108us/sample - loss: 0.0179 - val_loss: 0.0087
Epoch 83/100
3775/3775 [==============================] - 1s 133us/sample - loss: 0.0172 - val_loss: 0.0088
Epoch 84/100
3775/3775 [==============================] - 1s 190us/sample - loss: 0.0176 - val_loss: 0.0124
Epoch 85/100
3775/3775 [==============================] - 0s 102us/sample - loss: 0.0177 - val_loss: 0.0080
Epoch 86/100
3775/3775 [=====

In [18]:
train_GRU_model(xtrain, ytrain, xVal, yVal, filepath="./model_weights/2dPCA_GRU.h5")

Train on 3775 samples, validate on 755 samples
Epoch 1/100
3775/3775 [==============================] - 22s 6ms/sample - loss: 0.0797 - val_loss: 0.0133
Epoch 2/100
3775/3775 [==============================] - 1s 238us/sample - loss: 0.0407 - val_loss: 0.0085
Epoch 3/100
3775/3775 [==============================] - 1s 196us/sample - loss: 0.0355 - val_loss: 0.0292
Epoch 4/100
3775/3775 [==============================] - 1s 195us/sample - loss: 0.0318 - val_loss: 0.0065
Epoch 5/100
3775/3775 [==============================] - 1s 284us/sample - loss: 0.0301 - val_loss: 0.0296
Epoch 6/100
3775/3775 [==============================] - 1s 247us/sample - loss: 0.0305 - val_loss: 0.0053
Epoch 7/100
3775/3775 [==============================] - 1s 196us/sample - loss: 0.0266 - val_loss: 0.0284
Epoch 8/100
3775/3775 [==============================] - 1s 173us/sample - loss: 0.0278 - val_loss: 0.0068
Epoch 9/100
3775/3775 [==============================] - 1s 226us/sample - loss: 0.0268 - val_loss

3775/3775 [==============================] - 1s 180us/sample - loss: 0.0126 - val_loss: 0.0046
Epoch 77/100
3775/3775 [==============================] - 1s 175us/sample - loss: 0.0123 - val_loss: 0.0055
Epoch 78/100
3775/3775 [==============================] - 1s 167us/sample - loss: 0.0127 - val_loss: 0.0063
Epoch 79/100
3775/3775 [==============================] - 1s 162us/sample - loss: 0.0133 - val_loss: 0.0046
Epoch 80/100
3775/3775 [==============================] - 1s 151us/sample - loss: 0.0121 - val_loss: 0.0133
Epoch 81/100
3775/3775 [==============================] - 1s 154us/sample - loss: 0.0119 - val_loss: 0.0039
Epoch 82/100
3775/3775 [==============================] - 1s 171us/sample - loss: 0.0123 - val_loss: 0.0043
Epoch 83/100
3775/3775 [==============================] - 1s 149us/sample - loss: 0.0126 - val_loss: 0.0105
Epoch 84/100
3775/3775 [==============================] - 1s 154us/sample - loss: 0.0122 - val_loss: 0.0047
Epoch 85/100
3775/3775 [=================

### 2D2DPCA

In [19]:
xtrain, xVal, ytrain, yVal,sc,X_test,y_test = get_wt_2d2dpca_data(df)

/Users/fuyuqi/anaconda2/envs/notebook/lib/python3.7/site-packages/sklearn/utils/deprecation.py:66: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


In [20]:
train_LSTM_model(xtrain, ytrain, xVal, yVal, filepath="./model_weights/2d2dPCA_LSTM.h5")

Train on 3775 samples, validate on 755 samples
Epoch 1/100
3775/3775 [==============================] - 31s 8ms/sample - loss: 0.2360 - val_loss: 0.3111
Epoch 2/100
3775/3775 [==============================] - 1s 205us/sample - loss: 0.0592 - val_loss: 0.0250
Epoch 3/100
3775/3775 [==============================] - 1s 201us/sample - loss: 0.0266 - val_loss: 0.0120
Epoch 4/100
3775/3775 [==============================] - 1s 213us/sample - loss: 0.0220 - val_loss: 0.0072
Epoch 5/100
3775/3775 [==============================] - 1s 262us/sample - loss: 0.0199 - val_loss: 0.0088
Epoch 6/100
3775/3775 [==============================] - 1s 204us/sample - loss: 0.0185 - val_loss: 0.0043
Epoch 7/100
3775/3775 [==============================] - 1s 214us/sample - loss: 0.0179 - val_loss: 0.0060
Epoch 8/100
3775/3775 [==============================] - 1s 247us/sample - loss: 0.0184 - val_loss: 0.0037
Epoch 9/100
3775/3775 [==============================] - 1s 310us/sample - loss: 0.0168 - val_loss

3775/3775 [==============================] - 1s 154us/sample - loss: 0.0100 - val_loss: 0.0036
Epoch 77/100
3775/3775 [==============================] - 1s 165us/sample - loss: 0.0097 - val_loss: 0.0039
Epoch 78/100
3775/3775 [==============================] - 1s 182us/sample - loss: 0.0098 - val_loss: 0.0040
Epoch 79/100
3775/3775 [==============================] - 1s 279us/sample - loss: 0.0100 - val_loss: 0.0053
Epoch 80/100
3775/3775 [==============================] - 1s 213us/sample - loss: 0.0103 - val_loss: 0.0032
Epoch 81/100
3775/3775 [==============================] - 1s 230us/sample - loss: 0.0101 - val_loss: 0.0029
Epoch 82/100
3775/3775 [==============================] - 1s 386us/sample - loss: 0.0098 - val_loss: 0.0037
Epoch 83/100
3775/3775 [==============================] - 1s 286us/sample - loss: 0.0100 - val_loss: 0.0027
Epoch 84/100
3775/3775 [==============================] - 1s 279us/sample - loss: 0.0105 - val_loss: 0.0030
Epoch 85/100
3775/3775 [=================

In [21]:
train_RNN_model(xtrain, ytrain, xVal, yVal, filepath="./model_weights/2d2dPCA_RNN.h5")

Train on 3775 samples, validate on 755 samples
Epoch 1/100
3775/3775 [==============================] - 14s 4ms/sample - loss: 0.5628 - val_loss: 0.0130
Epoch 2/100
3775/3775 [==============================] - 0s 107us/sample - loss: 0.3962 - val_loss: 0.0228
Epoch 3/100
3775/3775 [==============================] - 0s 111us/sample - loss: 0.3296 - val_loss: 0.0134
Epoch 4/100
3775/3775 [==============================] - 0s 125us/sample - loss: 0.2879 - val_loss: 0.0105
Epoch 5/100
3775/3775 [==============================] - 0s 130us/sample - loss: 0.2403 - val_loss: 0.0105
Epoch 6/100
3775/3775 [==============================] - 0s 117us/sample - loss: 0.2078 - val_loss: 0.0133
Epoch 7/100
3775/3775 [==============================] - 0s 111us/sample - loss: 0.1741 - val_loss: 0.0168
Epoch 8/100
3775/3775 [==============================] - 0s 119us/sample - loss: 0.1618 - val_loss: 0.0116
Epoch 9/100
3775/3775 [==============================] - 0s 130us/sample - loss: 0.1321 - val_loss

Epoch 77/100
3775/3775 [==============================] - 0s 96us/sample - loss: 0.0159 - val_loss: 0.0061
Epoch 78/100
3775/3775 [==============================] - 0s 92us/sample - loss: 0.0152 - val_loss: 0.0062
Epoch 79/100
3775/3775 [==============================] - 0s 87us/sample - loss: 0.0151 - val_loss: 0.0059
Epoch 80/100
3775/3775 [==============================] - 0s 85us/sample - loss: 0.0148 - val_loss: 0.0065
Epoch 81/100
3775/3775 [==============================] - 0s 87us/sample - loss: 0.0158 - val_loss: 0.0070
Epoch 82/100
3775/3775 [==============================] - 0s 81us/sample - loss: 0.0147 - val_loss: 0.0062
Epoch 83/100
3775/3775 [==============================] - 0s 80us/sample - loss: 0.0154 - val_loss: 0.0058
Epoch 84/100
3775/3775 [==============================] - 0s 82us/sample - loss: 0.0154 - val_loss: 0.0107
Epoch 85/100
3775/3775 [==============================] - 0s 86us/sample - loss: 0.0154 - val_loss: 0.0056
Epoch 86/100
3775/3775 [=============

In [22]:
train_GRU_model(xtrain, ytrain, xVal, yVal, filepath="./model_weights/2d2dPCA_GRU.h5")

Train on 3775 samples, validate on 755 samples
Epoch 1/100
3775/3775 [==============================] - 16s 4ms/sample - loss: 0.1042 - val_loss: 0.0171
Epoch 2/100
3775/3775 [==============================] - 1s 150us/sample - loss: 0.0401 - val_loss: 0.0094
Epoch 3/100
3775/3775 [==============================] - 1s 156us/sample - loss: 0.0320 - val_loss: 0.0060
Epoch 4/100
3775/3775 [==============================] - 1s 142us/sample - loss: 0.0284 - val_loss: 0.0089
Epoch 5/100
3775/3775 [==============================] - 1s 146us/sample - loss: 0.0273 - val_loss: 0.0172
Epoch 6/100
3775/3775 [==============================] - 1s 145us/sample - loss: 0.0239 - val_loss: 0.0034
Epoch 7/100
3775/3775 [==============================] - 1s 148us/sample - loss: 0.0240 - val_loss: 0.0028
Epoch 8/100
3775/3775 [==============================] - 1s 137us/sample - loss: 0.0212 - val_loss: 0.0028
Epoch 9/100
3775/3775 [==============================] - 1s 139us/sample - loss: 0.0218 - val_loss

3775/3775 [==============================] - 1s 184us/sample - loss: 0.0101 - val_loss: 0.0039
Epoch 77/100
3775/3775 [==============================] - 1s 174us/sample - loss: 0.0108 - val_loss: 0.0039
Epoch 78/100
3775/3775 [==============================] - 1s 158us/sample - loss: 0.0103 - val_loss: 0.0027
Epoch 79/100
3775/3775 [==============================] - 1s 358us/sample - loss: 0.0114 - val_loss: 0.0029
Epoch 80/100
3775/3775 [==============================] - 1s 263us/sample - loss: 0.0100 - val_loss: 0.0067
Epoch 81/100
3775/3775 [==============================] - 1s 275us/sample - loss: 0.0106 - val_loss: 0.0051
Epoch 82/100
3775/3775 [==============================] - 1s 333us/sample - loss: 0.0104 - val_loss: 0.0029
Epoch 83/100
3775/3775 [==============================] - 1s 197us/sample - loss: 0.0109 - val_loss: 0.0090
Epoch 84/100
3775/3775 [==============================] - 1s 145us/sample - loss: 0.0101 - val_loss: 0.0039
Epoch 85/100
3775/3775 [=================